In [1]:
# LIBRARIES
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from sklearn.ensemble import IsolationForest

In [2]:
def read_data(file_name, header=0, sep=','):
    """
    Reads a csv file and returns a pandas dataframe
    """
    return pd.read_csv(file_name, sep=sep, header=header)

In [3]:
def remove_columns(df, columns):
    """
    Removes columns from a dataframe
    """
    return df.drop(columns, axis=1)

In [4]:
def remove_rows(df, rows):
    """
    Removes rows from a dataframe
    """
    return df.drop(rows, axis=0)

In [5]:
def remove_duplicates(df):
    """
    Removes duplicates from a dataframe
    """
    return df.drop_duplicates()

In [6]:
def convert_to_datatype(df, column, datatype):
    """
    Converts a column to a datatype
    """
    return df[column].astype(datatype)

In [7]:
def remove_outlier_numeric(df, column, min=-np.inf, max=np.inf):
    """
    Removes rows from a dataframe based on a condition
    """
    if min != -np.inf and max != np.inf:
        return df[(df[column] >= min) & (df[column] <= max)]
    elif min == -np.inf and max != np.inf:
        return df[(df[column] <= max)]
    elif min != -np.inf and max == np.inf:
        return df[(df[column] >= min)]

In [8]:
def detect_outliers_std(df, column, std=3):
    """
    Detect outliers based on standard deviation
    """
    
    # Set upper and lower limit to 3 standard deviation
    random_data_std = np.std(df[column])
    random_data_mean = np.mean(df[column])
    anomaly_cut_off = random_data_std * std
    
    lower_limit  = random_data_mean - anomaly_cut_off 
    upper_limit = random_data_mean + anomaly_cut_off

    # Generate outliers
    outliers = (df[column] < lower_limit) | (df[column] > upper_limit)
    return outliers

In [9]:
def detect_outliers_isolation_forest(df, column, contamination=0.1):
    """
    Detect outliers based on isolation forest
    """
    # Create isolation forest
    clf = IsolationForest(random_state=0, contamination=contamination)
    predictions = clf.fit_predict(df[column].to_numpy().reshape(-1, 1))
    return predictions == -1

In [10]:
def correct_category_levenshtein(df, column, correct_categories, threshold=80):
    """
    Corrects a column by using fuzzywuzzy to find the correct category
    """
    inconsistent_categories = pd.array(list(set(df[column].unique()) - set(correct_categories)))
    
    for inconsistent_category in inconsistent_categories:
        potential_match = process.extractOne(inconsistent_category, correct_categories)
        if potential_match[1] > threshold:
            df.loc[df[column] == inconsistent_category, column] = potential_match[0]
    return df

In [11]:
def convert_categorical_data(df, column):
    """
    Converts categorical data to numeric data
    """
    return pd.get_dummies(df[column], prefix=column)

In [48]:
def convert_categorical_data_numeric(df, column):
    df[column].replace(df[column].unique(), range(titanic_df['Name'].unique().shape[0]), inplace=True)
    return df

In [12]:
def fill_missing_values(df, column, value):
    """
    Fills missing values in a column with a value
    """
    return df[column].fillna(value)

In [13]:
def drop_missing_values(df, column):
    """
    Drops missing values in a column
    """
    return df.dropna(subset=[column])

In [14]:
def knn_impute(df, column):
    """
    Imputation using KNN
    """
    return df[column].fillna(df[column].mode()[0])

In [15]:
def pearson_spearman_correlation(df, column, output_column):
    """
    Calculates the pearson and spearman correlation between two columns
    """
    return df[column].corr(df[output_column], method='pearson'), df[column].corr(df[output_column], method='spearman')

# TESTING FUNCTIONS

In [25]:
titanic_df = read_data('../data/raw/titanic/titanic.csv', header=0)

In [17]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# correct_categories = ['male', 'female']

# titanic_df = correct_category_levenshtein(titanic_df, 'Sex', correct_categories, 80)

In [19]:
print(pearson_spearman_correlation(titanic_df, 'Fare', 'Survived'))

(0.2573065223849623, 0.32373613944480834)


In [20]:
# titanic_df[detect_outliers_isolation_forest(titanic_df, 'Fare', 0.1)].shape#.loc[x['Pclass'] == 1, :].shape

In [26]:
titanic_df.corr(method='pearson')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [28]:
convert_categorical_data(titanic_df, 'Name')

,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)","Name_Abelson, Mr. Samuel","Name_Abelson, Mrs. Samuel (Hannah Wizosky)","Name_Adahl, Mr. Mauritz Nils Martin","Name_Adams, Mr. John","Name_Ahlin, Mrs. Johan (Johanna Persdotter Larsson)","Name_Aks, Mrs. Sam (Leah Rosen)","Name_Albimona, Mr. Nassef Cassem",...,"Name_Yrois, Miss. Henriette (""Mrs Harbeck"")","Name_Zabour, Miss. Hileni","Name_Zabour, Miss. Thamine","Name_Zimmerman, Mr. Leo","Name_de Messemaeker, Mrs. Guillaume Joseph (Emma)","Name_de Mulder, Mr. Theodore","Name_de Pelsmaeker, Mr. Alfons","Name_del Carlo, Mr. Sebastiano","Name_van Billiard, Mr. Austin Blyler","Name_van Melkebeke, Mr. Philemon"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
888,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
test = titanic_df.copy()

In [73]:
test = convert_categorical_data_numeric(test, 'Name')
# test = convert_categorical_data_numeric(test, 'Age')
# test = convert_categorical_data_numeric(test, 'Ticket')
# test = convert_categorical_data_numeric(test, 'Cabin')
# test = convert_categorical_data_numeric(test, 'Embarked')
test.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,0,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,2,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,3,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,4,male,35.0,0,0,373450,8.0500,NaN,S


In [69]:
test['Age'].fillna(test['Age'].mode()[0], inplace=True)

In [75]:
test.corr(method='pearson')

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,1.000000,0.036186,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.005007,-0.052872,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.035144,-0.356187,0.083081,0.018443,-0.549500
Name,1.000000,-0.005007,-0.035144,1.000000,0.036186,-0.057527,-0.001652,0.012658
Age,0.036186,-0.052872,-0.356187,0.036186,1.000000,-0.232411,-0.155118,0.107554
SibSp,-0.057527,-0.035322,0.083081,-0.057527,-0.232411,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.001652,-0.155118,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.012658,0.107554,0.159651,0.216225,1.000000


In [74]:
test.corr(method='spearman')

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.034091,1.000000,0.044744,-0.061161,0.001235,-0.013975
Survived,-0.005007,1.000000,-0.339668,-0.005007,-0.023764,0.088879,0.138266,0.323736
Pclass,-0.034091,-0.339668,1.000000,-0.034091,-0.338558,-0.043019,-0.022801,-0.688032
Name,1.000000,-0.005007,-0.034091,1.000000,0.044744,-0.061161,0.001235,-0.013975
Age,0.044744,-0.023764,-0.338558,0.044744,1.000000,-0.135842,-0.209949,0.142980
SibSp,-0.061161,0.088879,-0.043019,-0.061161,-0.135842,1.000000,0.450014,0.447113
Parch,0.001235,0.138266,-0.022801,0.001235,-0.209949,0.450014,1.000000,0.410074
Fare,-0.013975,0.323736,-0.688032,-0.013975,0.142980,0.447113,0.410074,1.000000
